In [ ]:
!pip install Bio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import csv
from Bio import SeqIO
from sklearn.cluster import KMeans
from sklearn.cluster import MiniBatchKMeans
import numpy as np
import warnings


In [ ]:
sequences = []
max_seq_length = 0
for record in SeqIO.parse('/content/out_1 (1).fasta', 'fasta'):
    sequence = str(record.seq)
    sequences.append(sequence)
    seq_length = len(sequence)
    if seq_length > max_seq_length:
        max_seq_length = seq_length

In [ ]:
total_sequences = len(sequences)
print(f"Total Number of Sequences: {total_sequences}")

Total Number of Sequences: 200000


In [ ]:
n_sequences = len(sequences)
nucleotides = ['A', 'C', 'G', 'T']
nucleotide_map = {nucleotide: index for index, nucleotide in enumerate(nucleotides)}
encoded_sequences = np.full((n_sequences, max_seq_length), -1, dtype=int)
for i, sequence in enumerate(sequences):
    seq_length = len(sequence)
    for j, nucleotide in enumerate(sequence):
        index = nucleotide_map.get(nucleotide, -1)
        encoded_sequences[i, j] = index

In [ ]:
n_clusters = 500
batch_size = 1000  # Adjust the batch size based on available memory
with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=FutureWarning)
    kmeans = MiniBatchKMeans(n_clusters=n_clusters, random_state=42, batch_size=batch_size)
    cluster_labels = kmeans.fit_predict(encoded_sequences)

In [ ]:
clusters = {i: [] for i in range(n_clusters)}
for i, label in enumerate(cluster_labels):
    clusters[label].append(sequences[i])


In [ ]:
for cluster_id, sequences in clusters.items():
    cluster_size = len(sequences)
    print(f'Cluster {cluster_id + 1}:')
    for sequence in sequences:
        print(sequence)
    print(f'Sequence Count: {cluster_size}\n')

Streaming output truncated to the last 5000 lines.
GTGCTAACGTCCGTCGTGAAAAGGGAAACAACCCAGACCGTCAGCTAAGGTCCCAAAGTTATG
GTGCTAACGTCCGTCGTGAAAAGGGAAACAACCCAGACCGTCAGCTAAGGTCCCAAAGTTATG
GTGCTAACGTCCGTCGTGAAAAGGGAAACAACCCAGACCGTCAGCTAAGGTCCCAAAGTTATG
GTGCTAACGTCCGTCGTGAAAAGGGAAACAACCCAGACCGTCAGCTAAGGTCCCAAAGTTATG
GTGCTAACGTCCGTCGTGAAAAGGGAAACAACCCAGACCGTCAGCTAAGGTCCCAAAGTTATG
GTGCTAACGTCCGTCGTGAAAAGGGAAACAACCCAGACCGTCAGCTAAGGTCCCAAAGTTATG
GTGCTAACGTCCGTCGTGAAAAGGGAAACAACCCAGACCGTCAGCTAAGGTCCCAAAGTTATG
GTGCTAACGTCCGTCGTGAAAAGGGAAACAACCCAGACCGTCAGCTAAGGTCCCAAAGTTATG
GTGCTAACGTCCGTCGTGAAAAGGGAAACAACCCAGACCGTCAGCTAAGGTCCCAAAGTTATG
GTGCTAACGTCCGTCGTGAAAAGGGAAACAACCCAGACCGTCAGCTAAGGTCCCAAATTTATG
GTGCTAACGTCCGTCGTGAAAAGGGAAACAACCCAGACCGTCAGCTAAGGTCCCAAAGTTATG
GTGCAAAAATCACACATGAAAATCTAAACAACCAAGAAACTCAACTAACGACCCAAACTAATA
GTGCTAACGTCCGTCGTGAAAAGGGAAACAACCCAGACCGTCAGCTACGGTCCCAAAGTTATG
GTGCTAACGTCCGTCGTGAAAAGGGAAACAACCCAGACCGTCAGCTAAGGTCCCAAAGTTATG
GTGCTAACGTCCGTCGTGAAAAGGGAAACAACCCAGACCGTCAGCTAAGGTCC

In [ ]:
clstr_file = 'clusters.clstr'
with open(clstr_file, 'w') as f:
    cluster_id = 1
    for cluster, sequences in clusters.items():
        f.write(f'>Cluster {cluster_id}\n')
        for sequence in sequences:
            f.write(f'{sequence}\n')
        cluster_id += 1
        f.write('\n')

In [ ]:
with open('clusters.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Cluster Number', 'Sequence Count'])
    for cluster_id, sequences in clusters.items():
        cluster_size = len(sequences)
        writer.writerow([cluster_id + 1, cluster_size])

In [ ]:
sum_cluster_counts = sum(len(sequences) for sequences in clusters.values())
print(f"Sum of Cluster Sequence Counts: {sum_cluster_counts}")

Sum of Cluster Sequence Counts: 200000


In [ ]:
if total_sequences == sum_cluster_counts:
    print("Total number of sequences matches the sum of cluster sequence counts.")
else:
    print("Total number of sequences does not match the sum of cluster sequence counts.")

Total number of sequences matches the sum of cluster sequence counts.


In [ ]:
# Find the cluster with the highest number of sequences
max_cluster_id = max(clusters, key=lambda k: len(clusters[k])) + 1
max_cluster_sequences = clusters[max_cluster_id - 1]
max_cluster_sequence_count = len(max_cluster_sequences)

print(f"Cluster with the Highest Number of Sequences: {max_cluster_id}")
print(f"Number of Sequences in the Cluster: {max_cluster_sequence_count}")


Cluster with the Highest Number of Sequences: 15
Number of Sequences in the Cluster: 15670


In [16]:
subsequence = max_cluster_sequences[0][:14]
print(f"Subsequence of Length 14 from the Cluster: {subsequence}")

Subsequence of Length 14 from the Cluster: GGTGGCTGTAGTTT
